#### **Quantum Explorers 2023**
---
# Time Traveler バッジ - 量子エラー訂正と緩和 モジュール
#### **発展教材 - 量子エラー緩和手法と実装**
 [IBM Quantum Challenge Fall 2022, Lab 1 (和訳)](https://github.com/qiskit-community/ibm-quantum-challenge-fall-22/blob/main/content/lab-1/lab1-ja.ipynb) より作成しました。

### T-REX

T-REXは、 **Twirled Readout Error eXtinction** の略で、ゲートを"twirling"(トワリング、何回もくるくると作用)させる実装法です。ノイズを完全な回路実装の上に乗っている余分な確率的ゲート集合と見なすと、回路を実行するたびに、このノイズゲートの集合と「トワリング・セット」と呼ばれるゲートの集合からランダムに選ばれたゲートを結合させるという意味でトワリングと呼ぶことができます。このゲート集合をパウリ演算子の集合に選ぶと、「パウリ・トワリング」と呼ばれ、これは最もよく使われるトワリング・テクニックの1つです。

パウリ・トワリングはランダム化コンパイルの一種で、エンタングルゲートの前後にパウリゲート（I、X、Y、Z）のペアを挿入し、全体のユニタリーは同じままで、実装の仕方が異なるようにするものです。これにより、コヒーレントな誤差を確率的な誤差に変換し、十分な平均化によって除去することができます。これは、平均化の効果を得るために何度も行われます。注意：現在、すべての誤差を打ち消すのに十分な基底セットを使用しない場合があります。**[[1]](https://github.com/qiskit-research/qiskit-research/blob/main/docs/getting_started.ipynb)**

パウリ・トワリングについて詳しくは、[Nick Knows](https://www.youtube.com/watch?v=4MLHvmmpSQ8) のビデオをご覧ください。

options で `resilience_level=1` を設定すると、 `Estimator` 上でT-REXを有効にすることができます。 `resilience_level` が指定されていない場合、T-REXは実行時にデフォルトでアクティブになります。Qiskit Runtimeでは、T-REXを選択すると実行ごとに結果が変わることがあります。これは、トワリングに選択されるゲートセットが毎回異なるためで、結果がランダムな特性を持つ一因となっています。

T-REXについての詳細は [こちら](https://qiskit.org/ecosystem/ibm-runtime/locale/ja_JP/tutorials/Error-Suppression-and-Error-Mitigation.html#%E3%83%88%E3%83%AF%E3%82%A4%E3%83%AB%E8%AA%AD%E3%81%BF%E5%87%BA%E3%81%97%E3%82%A8%E3%83%A9%E3%83%BC%E6%B6%88%E5%8E%BB%E6%B3%95) をご覧ください。

### デジタルZNE（ゼロノイズ外挿）

デジタルZNE（Zero Noise Extrapolation）は、ノイズの多い量子コンピューターの誤差を、量子リソースを追加することなく軽減するための一般的な手法です。量子コンピューターのプログラムを変更し、異なるノイズレベルで計算を実行します。計算結果は、ノイズのない状態での推定値に外挿されます。

デジタルZNEでは、適切な値を求めるために、まず上記で説明したようにノイズをスケーリングします。スケーリングの方法には、アナログとデジタルの2種類があります。それぞれの方法の詳しい説明は下記で確認できます。どちらの方式が良いかは、まだ研究課題です。ここでのデジタルな方法は、物理的な量子パルスには作用させない手法です。

- パルス波の引き伸ばし（アナログ）： 同じパルス波を物理的により大きな時間に沿って引き伸ばしたものを回路に適用します。原理的には同じ回路だが、実効ノイズが大きくなります。 **[[2]](https://arxiv.org/pdf/1612.02058.pdf)**
- ローカル・フォールディング（デジタル）：入力回路をより多くのゲートでコンパイルします。各ゲート $G$ を $G$, $G^\dagger$, $G$ に置き換えます。シミュレーターでは回路に何もしませんが、実機を使用すると、ノイズが増加していくのがわかります。
- グローバル・フォールディング（デジタル）：ローカル・フォールディングと同じ操作ですが、このトリックを回路全体に適用します。

ノイズの多い期待値をゼロノイズ限界まで外挿する。これは一般的な回帰法で行われます。

デジタルZNE についての詳細は [こちら](https://qiskit.org/ecosystem/ibm-runtime/locale/ja_JP/tutorials/Error-Suppression-and-Error-Mitigation.html#%E3%82%BC%E3%83%AD%E3%83%BB%E3%83%8E%E3%82%A4%E3%82%BA%E5%A4%96%E6%8C%BF) をご覧ください。


### PEC

PEC（Probabilistic Error Cancellation、確率的エラーキャンセル）は、相関ノイズを捕らえることができるスパースノイズモデルを学習し、反転させることによってエラーを軽減するものです。PECは学習されたノイズモデルが緩和時に実際のノイズモデルを忠実に表す限り、バイアスのない期待値の推定値を返します。

PECは、学習したノイズを反転させる効果を模倣するために準確率法を使用します。これには、ユーザーの元の回路に関連するランダムな回路ファミリーからのサンプリングが必要です。PECを適用すると、入力回路と特性評価回路の両方で回路あたりのサンプル数も増やさない限り、返される期待値の推定値のばらつきが大きくなります。このばらつきに対処するために必要なサンプルの量は、緩和された回路のノイズ強度に応じて指数関数的に増加します。**[[3]](
)**

PEC についての詳細は [こちら](https://qiskit.org/ecosystem/ibm-runtime/locale/ja_JP/tutorials/Error-Suppression-and-Error-Mitigation.html#%E7%A2%BA%E7%8E%87%E7%9A%84%E3%82%A8%E3%83%A9%E3%83%BC%E3%83%BB%E3%82%AD%E3%83%A3%E3%83%B3%E3%82%BB%E3%83%AB) をご覧ください。


<div class="alert alert-block alert-info">
    
<h3>量子エラー緩和技術の実装</h3>

Qiskit runtime の resilience level は、エラーに対する回復力をどの程度構築するか、つまりエラー緩和のレベルを指定するための指標です。レベルが高いほど、処理時間が長くなる代わりに、より正確な結果が生成されます。Qiskit runtime でより高いレベルの弾力性を有効にすることができますが、 **この機能は現在ベータ版** であり、期待通りの結果が得られない可能性があることに注意してください！以下のコードブロックの指示に従って自由に試してみてください！

以下のリソースが便利です：

- [Qiskit ドキュメント: Options](https://qiskit.org/ecosystem/ibm-runtime/stubs/qiskit_ibm_runtime.options.Options.html)
- [エラー緩和の設定](https://qiskit.org/ecosystem/ibm-runtime/locale/ja_JP/how_to/error-mitigation.html)

In [ ]:
#import required libraries
import time
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
from qiskit.quantum_info import Statevector, Pauli, SparsePauliOp
from qiskit.circuit.library import RealAmplitudes
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
from qiskit.tools.visualization import plot_histogram
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator
from qiskit.providers.fake_provider import FakeManila
from qiskit_aer.noise import NoiseModel

# Import FakeBackend
fake_backend = FakeManila()
noise_model = NoiseModel.from_backend(fake_backend)

In [ ]:
# Save the Runtime account credentials if you have not done so already
# If you need to overwrite the account info, please add `overwrite=True`
# QiskitRuntimeService.save_account(channel='ibm_quantum', token='my_token', overwrite=True)

service = QiskitRuntimeService(channel="ibm_quantum", instance='ibm-q/open/main')

backend = service.backends(simulator=True)[0]
print(backend)

In [ ]:
# This is the circuit you used in the main syllabus for learning M3
theta = Parameter('theta')

qc = QuantumCircuit(2)
qc.x(1)
qc.h(0)
qc.cp(theta,0,1)
qc.h(0)

qc.draw()

次のセルのコードにあるように、 resilience level を値 `[1, 2, 3]` のいずれかに変更し、Estimator を実行して、resilience level に応じて出力がどのように変化するかを確認してください。

In [ ]:
phases = np.linspace(0, 2*np.pi, 50)
individual_phases = [[ph] for ph in phases]
ZZ = SparsePauliOp.from_list([("ZZ", 1)])

options = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },  
    resilience_level=0
)

options_with_em = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },  
    resilience_level=1 ### Change the value in this line ###
)

In [ ]:
with Session(service=service, backend=backend):    
    estimator = Estimator(options=options)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    param_results = job.result()
    exp_values = param_results.values
    
    estimator = Estimator(options=options_with_em)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    param_results = job.result()
    exp_values_with_em = param_results.values

In [ ]:
plt.plot(phases, exp_values, 'o', label='noisy')
plt.plot(phases, exp_values_with_em, 'o', label='mitigated')
plt.plot(phases, 2*np.sin(phases/2,)**2-1, label='theory')
plt.xlabel('Phase')
plt.ylabel('Expectation')
plt.legend();

<div class="alert alert-block alert-info">
    
<h3>オプション: 実機のバックエンドでの実行</h3>

上記のコードを **実機の量子バックエンド** で自由にテストして、QPU で実行したときに結果がどのように変化するかを確認してください。 Quantum Explorer の場合、`hub`, `group`, `project` が割り当てられています。 以下のセルを自由に設定して、結果を試してみてください。
    
</div> 

In [ ]:
# Get the least-busy backend, this step may take a while
real_backend = service.least_busy(min_num_qubits=2, simulator=False)

print(f"The least busy backend is {real_backend.name}.")

`Estimator` で実機のバックエンドを使用します:

In [ ]:
with Session(service=service, backend=real_backend):    
    estimator = Estimator(options=options)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    print(f"First job ID: {job.job_id()}")
    
    estimator = Estimator(options=options_with_em)
    job = estimator.run(circuits=[qc]*len(phases), parameter_values=individual_phases, observables=[ZZ]*len(phases))
    print(f"Second job ID: {job.job_id()}")

In [ ]:
#Get job results
job = service.job('place first job ID here')
param_results = job.result()
exp_values = param_results.values

job_with_em = service.job('place second job ID here')
param_results_with_em = job_with_em.result()
exp_values_with_em = param_results_with_em.values

Job の実行中、job を monitor できます。いずれかの実行に対して表示された job ID をコピーし、 **[ここ](https://quantum-computing.ibm.com/jobs)** の検索バーに入力するだけです。このページにアクセスするには、IBM Quantum Platform にログインしていることを確認してください。

In [ ]:
plt.plot(phases, exp_values, 'o', label='noisy')
plt.plot(phases, exp_values_with_em, 'o', label='mitigated')
plt.plot(phases, 2*np.sin(phases/2,)**2-1, label='theory')
plt.xlabel('Phase')
plt.ylabel('Expectation')
plt.legend();

# 参考文献
- [1] **[Getting Started - Qiskit Research](https://github.com/qiskit-research/qiskit-research/blob/main/docs/getting_started.ipynb)**
- [2] **[Error mitigation for short-depth quantum circuits - Kristan Temme, Sergey Bravyi and Jay M. Gambetta](https://arxiv.org/pdf/1612.02058.pdf)**
- [3] **[Configure error mitigation](https://qiskit.org/ecosystem/ibm-runtime/locale/ja_JP/how_to/error-mitigation.html)**

Translated and adapted by: Kifumi Numata

In [1]:
from qiskit.tools.jupyter import *
%qiskit_version_table